In [1]:
import numpy as np
import torch
import tensorflow as tf
from HelperClasses import Direction, Point, Board, Snake
from SnakeGameClass import SnakeGame
from SnakeGameClass_4Directions import SnakeGame as SnakeGame_4D
import tensorflow as tf

c:\Users\otto\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
game = SnakeGame_4D(5,5)
# game.snake
snake = Snake((5,5))
# print(snake.body)
game.state

array([[0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ],
       [0.5, 0.5, 1. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. ]])

In [5]:
game.state.shape

(5, 5)

: 

In [25]:
board = Board(20,20,snake=snake, food=game.food)
board

     0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4   1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
5   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5  0.5  0.5  0.0  0.0  0.0  0.0   
11  0.0  0.0  0.0  0.0  0.0 

In [4]:
# board.get_available_space_3directions(Direction.UP)

In [26]:
b_t = board.to_tensor(channels=1)
# b2_t = b2.to_tensor(channels=1)
# b_tf = board.to_tensorflow_tensor()

# b_tf_stacked = tf.stack((b_tf,b_tf,b_tf,b_tf), axis=-1)
# b_tf_stacked = tf.stack((b_tf,b_tf), axis=-1)
# b_tf_stacked = tf.expand_dims(b_tf_stacked, axis=0)
print(b_t.shape)
# print(b2_t.shape)
b_t

torch.Size([20, 20])


tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0

In [27]:
stacked = torch.stack((b_t,b_t,b_t,b_t)).unsqueeze(0)
stacked.shape

torch.Size([1, 4, 20, 20])

In [28]:
channels = 2
net = torch.nn.Sequential(
    torch.nn.Conv2d(4, 32, (3,3)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((2, 2)),
    torch.nn.Conv2d(32, 64, (2,2)),
    # torch.nn.ReLU(),
    # torch.nn.Flatten(),
    # torch.nn.Linear(64*3*3,64*2*2),
    # torch.nn.ReLU(),
    # torch.nn.Linear(64*2*2,4),
)

tf_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64, 2, activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64*2*2, activation="relu"),
    tf.keras.layers.Dense(4, activation="linear")
    
])




In [29]:
# out = net(tens)
out = net(stacked)
print(out.shape)
# tf_model.layers[0](b_tf_stacked)
# out.shape
# print(b_tf_stacked.shape)
# out = tf_model(b_tf_stacked)
# print(out.shape)
# tf.argmax(out[0]).numpy()

torch.Size([1, 32, 18, 18])


In [ ]:
temp = torch.stack((stacked[0],stacked[0]))

temp.shape


net(temp)

tensor([[-0.0742,  0.0941,  0.0351],
        [-0.0742,  0.0941,  0.0351]], grad_fn=<AddmmBackward0>)